# NASDAQ LSTM Hyperparameter Tuning

This notebook implements automated hyperparameter optimization for LSTM neural networks applied to NASDAQ time series prediction using Optuna framework.

## Dependencies and Imports

Essential libraries for data processing, machine learning, and hyperparameter optimization.

In [1]:
%%capture
%pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import optuna
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

/Users/isha/Desktop/NYU/Bootcamp/Bootcamp-4/homework/Repo/bootcamp_isha_patro/bootcamp_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Loading and Preprocessing

Load NASDAQ historical data and prepare it for time series modeling with normalization.

In [3]:
df = pd.read_csv('data/HistoricalData.csv')
df.rename(columns={'Close/Last':'Close'}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'])
df.sort_values(by='Date', ascending=True, inplace=True)
series = pd.to_numeric(df['Close'], errors='coerce').dropna().values.reshape(-1,1)
scaler = MinMaxScaler()
scaled = scaler.fit_transform(series)

## Sequence Creation Utility

Function to create input-output sequences for LSTM training with configurable lookback window.

In [4]:
def make_sequences(arr, look_back):
    X, y = [], []
    for i in range(look_back, len(arr)):
        X.append(arr[i-look_back:i])
        y.append(arr[i])
    X = np.array(X)
    y = np.array(y)
    return X, y

## Model Training and Evaluation Function

Comprehensive function that builds, trains, and evaluates LSTM models with given hyperparameters.

In [5]:
train_ratio = 0.8

def train_eval(params):
    look_back = params['look_back']
    X, y = make_sequences(scaled, look_back)
    split = int(len(X)*train_ratio)
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]
    
    model = Sequential()
    model.add(LSTM(params['lstm_units'], input_shape=(look_back,1), kernel_regularizer=l2(params['l2_reg']), return_sequences=False))
    if params['dropout_rate']>0:
        model.add(Dropout(params['dropout_rate']))
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=params['learning_rate']), loss='mse')
    
    es = EarlyStopping(monitor='val_loss', patience=params['patience'], restore_best_weights=True)
    model.fit(X_train, y_train, epochs=params['epochs'], batch_size=params['batch_size'], 
              validation_split=params['validation_split'], shuffle=params['shuffle'], 
              callbacks=[es], verbose=0)
    
    train_pred = model.predict(X_train, verbose=0)
    test_pred = model.predict(X_test, verbose=0)
    train_actual = scaler.inverse_transform(y_train)
    test_actual = scaler.inverse_transform(y_test)
    train_pred_actual = scaler.inverse_transform(train_pred)
    test_pred_actual = scaler.inverse_transform(test_pred)
    
    train_rmse = float(np.sqrt(mean_squared_error(train_actual, train_pred_actual)))
    train_mae = float(mean_absolute_error(train_actual, train_pred_actual))
    train_r2 = float(r2_score(train_actual, train_pred_actual))
    test_rmse = float(np.sqrt(mean_squared_error(test_actual, test_pred_actual)))
    test_mae = float(mean_absolute_error(test_actual, test_pred_actual))
    test_r2 = float(r2_score(test_actual, test_pred_actual))
    
    return {'train_rmse':train_rmse,'train_mae':train_mae,'train_r2':train_r2,
            'test_rmse':test_rmse,'test_mae':test_mae,'test_r2':test_r2}

## Baseline Model Evaluation

Establish baseline performance with default hyperparameters before optimization.

In [6]:
baseline = {'look_back':20,'lstm_units':16,'dropout_rate':0.2,'l2_reg':0.0005,
           'learning_rate':0.001,'epochs':50,'batch_size':32,'patience':8,
           'validation_split':0.2,'shuffle':False}
before = train_eval(baseline)

2025-10-25 11:09:09.732836: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1
2025-10-25 11:09:09.732868: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2025-10-25 11:09:09.732874: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2025-10-25 11:09:09.733579: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-10-25 11:09:09.733814: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2025-10-25 11:09:10.638046: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:09:10.795812: I te

## Optuna Objective Function

Define the optimization objective function with hyperparameter search spaces for Optuna.

In [7]:
def objective(trial):
    params = {
        'look_back': trial.suggest_int('look_back', 15, 30),
        'lstm_units': trial.suggest_int('lstm_units', 16, 64),
        'dropout_rate': trial.suggest_float('dropout_rate', 0.0, 0.4),
        'l2_reg': trial.suggest_float('l2_reg', 0.0, 0.001),
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 5e-3, log=True),
        'epochs': trial.suggest_int('epochs', 30, 80),
        'batch_size': trial.suggest_categorical('batch_size', [16,32,64]),
        'patience': trial.suggest_int('patience', 5, 12),
        'validation_split': trial.suggest_float('validation_split', 0.1, 0.3),
        'shuffle': False
    }
    metrics = train_eval(params)
    return metrics['test_r2']

## Hyperparameter Optimization

Execute Optuna study to find optimal hyperparameters maximizing test R² score.

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=60, show_progress_bar=True)
best_params = study.best_params
best_params['shuffle']=False
metrics_after = train_eval(best_params)

[I 2025-10-25 11:09:28,711] A new study created in memory with name: no-name-aad041fd-4970-4bc1-b199-280b27dca53f
  0%|          | 0/60 [00:00<?, ?it/s]WARNING:absl:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.
2025-10-25 11:09:29.352543: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:09:29.510049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:09:29.627077: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:09:30.513865: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:09:30.565532: I ten

[I 2025-10-25 11:09:52,887] Trial 0 finished with value: 0.8010929146469492 and parameters: {'look_back': 30, 'lstm_units': 61, 'dropout_rate': 0.26224256298077964, 'l2_reg': 0.0007240564985022812, 'learning_rate': 0.00021038264151354466, 'epochs': 47, 'batch_size': 32, 'patience': 12, 'validation_split': 0.1644648709373222}. Best is trial 0 with value: 0.8010929146469492.


2025-10-25 11:09:53.472979: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:09:53.605674: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:09:53.719470: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:09:54.442148: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:09:54.496001: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:10:12.139449: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:10:12.189927: I tensorflow/core/grappler/optimizers/cust

[I 2025-10-25 11:10:12,503] Trial 1 finished with value: 0.9036188170510455 and parameters: {'look_back': 16, 'lstm_units': 19, 'dropout_rate': 0.3352922668334735, 'l2_reg': 0.0009421746729220679, 'learning_rate': 0.00011072341954206338, 'epochs': 55, 'batch_size': 64, 'patience': 8, 'validation_split': 0.25934013426292724}. Best is trial 1 with value: 0.9036188170510455.


2025-10-25 11:10:13.124645: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:10:13.267653: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:10:13.380072: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:10:14.222663: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:10:14.276703: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:10:22.643339: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:10:22.684735: I tensorflow/core/grappler/optimizers/cust

...

2025-10-25 11:12:36.383817: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:12:36.513793: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:12:36.641508: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:12:37.232679: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:12:37.283386: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:12:49.298957: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:12:49.340879: I tensorflow/core/grappler/optimizers/cust

[I 2025-10-25 11:12:49,675] Trial 10 finished with value: 0.8121685332324335 and parameters: {'look_back': 22, 'lstm_units': 32, 'dropout_rate': 0.15678446411244892, 'l2_reg': 0.0006270839467154427, 'learning_rate': 0.0006243849132095543, 'epochs': 37, 'batch_size': 64, 'patience': 11, 'validation_split': 0.24620403892268117}. Best is trial 5 with value: 0.9560507071795832.


2025-10-25 11:12:50.774084: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:12:50.927507: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:12:51.080715: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:12:52.440994: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:12:52.498798: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:13:12.233612: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:13:12.285426: I tensorflow/core/grappler/optimizers/cust

[I 2025-10-25 11:13:12,704] Trial 11 finished with value: 0.8794749449252731 and parameters: {'look_back': 25, 'lstm_units': 43, 'dropout_rate': 0.1990265267148536, 'l2_reg': 2.9956472995096116e-05, 'learning_rate': 0.000646615406069276, 'epochs': 30, 'batch_size': 16, 'patience': 10, 'validation_split': 0.23588140390066156}. Best is trial 5 with value: 0.9560507071795832.


2025-10-25 11:13:13.317047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:13:13.456841: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:13:13.622831: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:13:15.257337: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-25 11:13:15.315875: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


## Results Comparison

Compare baseline and optimized model performance across all evaluation metrics.

In [ ]:
results = pd.DataFrame([dict(experiment='before',**before), 
                       dict(experiment='after',**metrics_after)])
results